In [41]:
using Combine.Util
using Combine.Transformers
import RDatasets

In [3]:
iris = RDatasets.dataset("datasets", "iris")
X = convert(Array, iris[[:SepalLength, :SepalWidth, :PetalLength, :PetalWidth]])
y = convert(Array, iris[:Species]);

# Split into training and test sets
(train_ind, test_ind) = holdout(size(X, 1), 0.3)

([112,59,103,31,84,135,129,108,60,58  …  75,46,93,4,92,73,102,77,100,38],[53,44,144,69,98,74,48,97,34,120  …  121,3,101,149,12,36,35,125,71,28])

In [4]:
prunedTreeLearner = PrunedTree()

Combine.Transformers.DecisionTreeWrapper.PrunedTree(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:impl_options,Dict(:purity_threshold=>1.0)),Pair{Symbol,Any}(:output,:class)))

In [5]:
pipeline = Pipeline(Dict(
        :transformers => [
            OneHotEncoder(), # Encodes nominal features into numeric
            Imputer(), # Imputes NA values
            #StandardScaler(), # Standardizes features 
            prunedTreeLearner # Predicts labels on instances
        ]
    ))

Combine.Transformers.CombineTransformers.Pipeline(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:transformers,Combine.Types.Transformer[Combine.Transformers.CombineTransformers.OneHotEncoder(nothing,Dict(:nominal_columns=>nothing,:nominal_column_values_map=>nothing)),Combine.Transformers.CombineTransformers.Imputer(nothing,Dict(:strategy=>mean)),Combine.Transformers.DecisionTreeWrapper.PrunedTree(nothing,Dict{Symbol,Any}(Pair{Symbol,Any}(:impl_options,Dict(:purity_threshold=>1.0)),Pair{Symbol,Any}(:output,:class)))]),Pair{Symbol,Any}(:transformer_options,nothing)))

In [6]:
# Train
fit!(pipeline, X[train_ind, :], y[train_ind]);

In [7]:
# Predict
predictions = transform!(pipeline, X[test_ind, :]);

In [8]:
sum(predictions .== y[test_ind])/length(predictions)*100

93.33333333333333

In [9]:
result = score(:accuracy, y[test_ind], predictions)
println(result)

93.33333333333333


In [10]:
function processModel(learner)
    iris = RDatasets.dataset("datasets", "iris")
    X = convert(Array, iris[[:SepalLength, :SepalWidth, :PetalLength, :PetalWidth]])
    y = convert(Array, iris[:Species]);
    (train_ind, test_ind) = holdout(size(X, 1), 0.3)
    pipeline = Pipeline(Dict(
            :transformers => [
                OneHotEncoder(), # Encodes nominal features into numeric
                Imputer(), # Imputes NA values
                #StandardScaler(), # Standardizes features 
                learner # Predicts labels on instances
            ]
        ))
    # Train
    fit!(pipeline, X[train_ind, :], y[train_ind]);
    # Predict
    predictions = transform!(pipeline, X[test_ind, :]);
    result = score(:accuracy, y[test_ind], predictions)
    return(result)
end

processModel (generic function with 1 method)

In [12]:
adaLearner = DecisionStumpAdaboost(Dict(
  # Output to train against
  # (:class).
  :output => :class,
  # Options specific to this implementation.
  :impl_options => Dict(
    # Number of boosting iterations.
    :num_iterations => 7
  )
))
processModel(adaLearner)

86.66666666666667

In [13]:
rfLearner = RandomForest(Dict(
  :output => :class,
  :impl_options => Dict(
    :num_subfeatures => nothing,
    :num_trees => 10,
    :partial_sampling => 0.7
  )
))
processModel(rfLearner)

97.77777777777777

In [15]:
using ScikitLearn
@sk_import neighbors: KNeighborsClassifier
@sk_import svm: SVC

skLearner = SKLLearner(Dict(
  :output => :class,
  #:learner => "KNeighborsClassifier",
  :learner => "SVC",
  :impl_options => Dict()
))
processModel(skLearner)

97.77777777777777

In [16]:
voteLearner = VoteEnsemble(Dict(
  :output => :class,
  # Learners in voting committee.
  :learners => [RandomForest(),PrunedTree(), DecisionStumpAdaboost()]
))
processModel(voteLearner)

95.55555555555556

In [18]:
bestLearner = BestLearner(Dict(
  :output => :class,
  :partition_generator => (X, y) -> kfold(size(X, 1), 5),
  :selection_function => (learner_partition_scores) -> findmax(mean(learner_partition_scores, 2))[2],      
  :score_type => Real,
  :learners => [PrunedTree(), DecisionStumpAdaboost(), RandomForest()],
  :learner_options_grid => nothing
))
processModel(bestLearner)

97.77777777777777

In [39]:
stackLearner = StackEnsemble(Dict(
  :output => :class,
  :learners => [PrunedTree(), DecisionStumpAdaboost(), RandomForest(),skLearner,voteLearner,bestLearner],
  :stacker => RandomForest(),
  # Proportion of training set left to train stacker itself.
  :stacker_training_proportion => 0.3,
  :keep_original_features => false
))
processModel(stackLearner)

95.55555555555556

In [40]:
results=@parallel (vcat) for i=1:30
   processModel(stackLearner)
end
println("acc = ",round(mean(results))," +/- ",round(std(results)))

acc = 95.0 +/- 3.0


In [27]:
results

30-element Array{Float64,1}:
 100.0   
 100.0   
  95.5556
 100.0   
  91.1111
 100.0   
  95.5556
  97.7778
  97.7778
  95.5556
  97.7778
  97.7778
  97.7778
   ⋮     
  97.7778
  91.1111
  95.5556
  97.7778
  97.7778
  95.5556
  93.3333
 100.0   
  97.7778
  91.1111
  95.5556
  97.7778

In [17]:
#svmcrt = CRTLearner(Dict(
  # Output to train against
  # (:class).
  #:output => :class,
  #:learner => "rf",
  #:learner => "svmLinear2",
  #:learner => "rpart",
  #:learner => "lda",
  #:impl_options => Dict()
#))